### Финальный результат (версия для параграфов)

In [ ]:
# импортируем необходимые библиотеки

from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

import numpy as np

from sentence_transformers import SentenceTransformer, util
import statistics as stat

In [ ]:
# загрузка данных из файла

def load_data_from_docx(file_name):
    document = Document(file_name)
    words_to_delete = []
    for para in document.paragraphs:
        if para.style.name.startswith('Heading') and para.text.strip():
            words_to_delete.append(para.text)
    words_to_delete.append('Таблица')
    rez,found_start=[],False
    for paragraph in document.paragraphs:
        if words_to_delete[0] in paragraph.text:
            found_start = True
        elif found_start:
            rez.append(paragraph.text)
    return [i for i in rez if i.strip() and not any(word in i for word in words_to_delete)]

In [ ]:
real_parts=load_data_from_docx("П-07.22-6.1-КР-ТЧ.docx")
#real_parts=load_data_from_docx("ПР_12_20_ПД_ПОС_Проект_организации_строительства.docx_review.docx")
#len(real_parts)
#real_parts
data = real_parts

In [ ]:
# расстояние каждого предложения до других (значения + сортировка)

def generate_seq_of_similarity_inc(data:list):
    results,results_values=[],[]
    embedder = SentenceTransformer('distiluse-base-multilingual-cased')
    sentence_embeddings = embedder.encode(data, convert_to_tensor=True)
    for i in range(len(data)):
        SENTENCE_NUM=i
        given_sentence_embedding = sentence_embeddings[SENTENCE_NUM]
        cosine_similarities_bert = util.pytorch_cos_sim(given_sentence_embedding.reshape(1, -1), sentence_embeddings).numpy().flatten()
        results_values.append(cosine_similarities_bert)
        results.append(cosine_similarities_bert.argsort())
    return results,results_values

In [ ]:
results,results_values=generate_seq_of_similarity_inc(data)

In [ ]:
# поиск "непохожих" предложений

def get_frequence(results,results_values,isPrint=False):
    sorts=[]
    for j in range(len(results)):
        sorts.append([i for i in results[j] if results_values[j][i]<np.mean(results_values)])
    flat_arr = [val for arr in sorts for val in arr]
    freq_dict = {val: flat_arr.count(val) for val in set(flat_arr)}
    sorted_freq_list=sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
    global data
    data_stat=[i[1] for i in sorted_freq_list]
    find,mark=[],False
    if isPrint:
        print("\nЧастота встречаемости элементов (статистический анализ):")
        mark=True
    for val, count in sorted_freq_list: 
        if count >= round(stat.mean(data_stat)+1.9*stat.stdev(data_stat)):
            if mark:
                print(f"{val}: {count}")
                print(data[val])
            find.append(data[val])
            
    return sorted_freq_list,find

In [ ]:
sorted_freq_list,find=get_frequence(results,results_values,False)
#sorted_freq_list
find

In [ ]:
# выгрузка результатов в отдельный файл

def get_review(file_name,find):
    document = Document(file_name)
    # Список предложений, которые нужно выделить
    sentences_to_highlight = find
    # Проходим по всем абзацам в документе
    for paragraph in document.paragraphs:
        # Проверяем, содержит ли абзац какое-либо предложение из списка
        for sentence in sentences_to_highlight:
            if sentence in paragraph.text:
                # Если предложение найдено, выделяем его
                for run in paragraph.runs:
                    if run.text in sentence:
                        #run.font.color.rgb = RGBColor(255, 0, 0)
                        run.font.highlight_color = WD_COLOR_INDEX.YELLOW

    return document.save(f'{file_name}_review.docx')

In [ ]:
get_review("П-07.22-6.1-КР-ТЧ.docx",find)

#### Все в одной функции

In [ ]:
# импортируем необходимые библиотеки

from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

import numpy as np

from sentence_transformers import SentenceTransformer, util
import statistics as stat
import re
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from razdel import sentenize, tokenize

####  тестовые версии функции load_data_from_docx

In [ ]:
#загрузка данных из файла

def load_data_from_docx(file_name):
    document = Document(file_name)
    words_to_delete = []
    for para in document.paragraphs:
        if para.style.name.startswith('Heading') and para.text.strip():
            words_to_delete.append(para.text)
    words_to_delete.append('Таблица')
    rez,found_start=[],False
    for paragraph in document.paragraphs:
        if words_to_delete[0] in paragraph.text:
            found_start = True
        elif found_start:
            rez.append(paragraph.text)
    return [i for i in rez if i.strip() and not any(word in i for word in words_to_delete)]

In [ ]:
#загрузка данных из файла

def load_data_from_docx(file_name):
    document = Document(file_name)
    words_to_delete = []
    for para in document.paragraphs:
        if para.style.name.startswith('Heading') and para.text.strip():
            words_to_delete.append(para.text)
    bold_under_to_delete = []

    #кусок по выюорке подзаголовков

    for paragraph in document.paragraphs:
         paragraph_text = ""
         for run in paragraph.runs:
             if run.bold and run.underline:
                 paragraph_text += run.text
         if paragraph_text:
             bold_under_to_delete.append(paragraph_text)


    bold_under_to_delete.append("Таблица")

    #проверка по регуляркам

    def split_sentences(text):
        sentences = re.split(r'(?<!г)\.(?=\s+[А-Я])(?=(?:[^«»]*«[^»]*»)*[^«»]*$)(?<!\d\.)', text)
        return sentences
    
    rez,rez_heads,found_start=[],[],False
    for paragraph in document.paragraphs:
        if any(word in paragraph.text for word in words_to_delete):
            found_start = True
            if rez_heads:
                rez.append(rez_heads)
                rez_heads = []
            
            #добавление заголовка

            para_sentences = split_sentences(paragraph.text)
            for sentence in para_sentences:
                sentence = sentence.strip()
                if sentence and (sentence not in bold_under_to_delete and any(word not in sentence for word in bold_under_to_delete)):
                    rez_heads.append(sentence)

        elif found_start:
            para_sentences = split_sentences(paragraph.text)
            #para_sentences =paragraph.text.split('. ')
            for sentence in para_sentences:
                sentence = sentence.strip()
                if sentence and (sentence not in bold_under_to_delete and any(word not in sentence for word in bold_under_to_delete)):
                #if sentence and "Таблица" not in sentence:
                    rez_heads.append(sentence)

    if rez_heads:
        rez.append(rez_heads)
        
    return rez

#### итоговая функция load_data_from_docx

In [ ]:
# загрузка данных из файла

def load_data_from_docx(file_name):
    document = Document(file_name)
    words_to_delete = []
    for para in document.paragraphs:
        if para.style.name.startswith('Heading') and para.text.strip():
            words_to_delete.append(para.text)
    bold_under_to_delete = []

    # кусок по выборке подзаголовков

    # for paragraph in document.paragraphs:
    #     paragraph_text = ""
    #     for run in paragraph.runs:
    #         if run.bold and run.underline:
    #             paragraph_text += run.text
    #     if paragraph_text:
    #         bold_under_to_delete.append(paragraph_text)

    bold_under_to_delete.append("Таблица")

    # проверка по регуляркам
    #def split_sentences(text):
        #sentences = re.split(r'(?=(?:[^«»]*«[^»]*»)*[^«»]*$)', text)
            #return sentences
        #sentences = re.split(r'(?<!\b(?:т\.д|т\.п|р\.п))\.(?=[А-Я])\.(?<!г)\.(?=[А-Яа-я])(?=(?:[^«»]*«[^»]*»)*[^«»]*$)(?<!\d\.)',text)

    # def split_sentences(text):
    #      sentences =list(sentenize(text))
    #      return [i.text for i in sentences]

    def split_sentences(text):
        sentences = list(sentenize(text))
        sentence_texts = [i.text for i in sentences]
    
        combined_sentences = []
        temp_sentence = ""
    
        for i, sentence in enumerate(sentence_texts):
            temp_sentence += sentence
        
            # Проверка наличия непарных кавычек в предложении
            open_quotes = temp_sentence.count('"') % 2
        
            if open_quotes == 0:
                combined_sentences.append(temp_sentence.strip())
                temp_sentence = ""
            else:
                temp_sentence += " "
    
        # Если в конце осталась незакрытая кавычка
        if temp_sentence:
            combined_sentences.append(temp_sentence.strip())
    
        return combined_sentences
        
        
    # раскрытие заголовков
    def add_data_from_paragraph(rez_heads):
                para_sentences = split_sentences(paragraph.text)
                for sentence in para_sentences:
                    sentence = sentence.strip()
                    if sentence and (sentence not in bold_under_to_delete and any(word not in sentence for word in bold_under_to_delete)):
                        rez_heads.append(sentence)
                return rez_heads
    
    rez,rez_heads,found_start=[],[],False
    for paragraph in document.paragraphs:
        if any(word in paragraph.text for word in words_to_delete):
            found_start = True
            if rez_heads:
                rez.append(rez_heads)
                rez_heads = []
            
            # добавление заголовка
            rez_heads=add_data_from_paragraph(rez_heads)

        elif found_start:
            rez_heads=add_data_from_paragraph(rez_heads)
    
    if rez_heads:
        rez.append(rez_heads)
        
    return rez

In [ ]:
real_parts=load_data_from_docx("П-07.22-6.1-КР-ТЧ.docx")
data=real_parts

In [ ]:
# расстояние каждого предложения до других (значения + сортировка)

def generate_seq_of_similarity_inc(data:list):
    results,results_values=[],[]
    embedder = SentenceTransformer('distiluse-base-multilingual-cased')
    sentence_embeddings = embedder.encode(data, convert_to_tensor=True)
    for i in range(len(data)):
        SENTENCE_NUM=i
        given_sentence_embedding = sentence_embeddings[SENTENCE_NUM]
        cosine_similarities_bert = util.pytorch_cos_sim(given_sentence_embedding.reshape(1, -1), sentence_embeddings).numpy().flatten()
        results_values.append(cosine_similarities_bert)
        #results.append(cosine_similarities_bert.argsort())
    data_outliner=np.mean(results_values)+1.9*np.std(results_values)


    for i, lst in enumerate(results_values):
        lst = lst
        mask = (lst > data_outliner) & (lst < 0.99)
        lst = lst[~mask]  
        results_values[i] = np.array(lst)
        results.append(np.argsort(lst))
   

    # for j, (i, _) in enumerate(zip(results, results_values)):
    #     for item in i:
    #         if _[item] > data_outliner and _[item] < 0.99:
    #             mask = np.isin(i, item)
    #             i = np.array(i[~mask])
    #             mask2=np.isin(_,_[item])
    #             _ = np.array(_ [~mask2])
    #     results[j] = i
    #     results_values[j] = _
    
    return results,results_values

In [ ]:
res=[]
for i in data:
    res.append(generate_seq_of_similarity_inc(i))

In [ ]:
def get_frequence(results, results_values, data, isPrint=False):
    if len(results) == 1:
        print(results)
        sorts = [i for i in results[0] if results_values[0][i] < np.median(results_values[0])]
        freq_dict = {val: sorts.count(val) for val in set(sorts)}
        sorted_freq_list = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
        data_stat = [i[1] for i in sorted_freq_list]
        if len(data_stat) == 0:
            threshold = 0
        else:
            threshold = round(stat.mean(data_stat) + 1.9 * stat.stdev(data_stat))
    else:
        sorts = []
        median = np.median(np.concatenate([np.array(inner) for inner in results_values]))
        for j in range(len(results)):
            sorts.append([i for i in results[j] if results_values[j][i] < median])
        flat_arr = [val for arr in sorts for val in arr]
        freq_dict = {val: flat_arr.count(val) for val in set(flat_arr)}
        sorted_freq_list = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
        data_stat = [i[1] for i in sorted_freq_list]
        threshold = round(stat.mean(data_stat) + 1.9 * stat.stdev(data_stat))

    find, mark = [], False
    if isPrint:
        print("\nЧастота встречаемости элементов (статистический анализ):")
        mark = True
    for val, count in sorted_freq_list: 
        if count >= threshold:
            if mark:
                print(f"{val}: {count}")
                print(data[val])
            find.append(data[val])
    print(sorted_freq_list)
    print(threshold)
    return find

In [ ]:
find=[]
for i in range(len(res)):
    #print(len(res[i][0]),len(res[i][1]))
    find.append(get_frequence(res[i][0],res[i][1],data[i],isPrint=True))

find=[item for sublist in find for item in sublist]

In [ ]:
from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

def get_review(file_name, find):
    document = Document(file_name)
    sentences_to_highlight = find

    for paragraph in document.paragraphs:
        for sentence in sentences_to_highlight:
            if sentence in paragraph.text:
                # Find the start and end indices of the sentence in the paragraph text

                # Create a new run for the highlighted sentence
                run = paragraph.add_run(sentence)
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.font.highlight_color = WD_COLOR_INDEX.YELLOW

                # Remove the original text from the paragraph
                #paragraph.text = paragraph.text.replace(sentence, '')

    document.save(f'{file_name}_review.docx')

In [ ]:
get_review("П-07.22-6.1-КР-ТЧ.docx",find)

In [ ]:
# итоговая функция

def get_non_simillar_sentences(filename,see_outliners=False):
    data = load_data_from_docx(filename)
    res,find=[],[]
    for i in data:
        res.append(generate_seq_of_similarity_inc(i))
        
    for i in range(len(res)):
        find.append(get_frequence(res[i][0],res[i][1],data[i],isPrint=True))

    find=[item for sublist in find for item in sublist]
    print(find)
    get_review(filename,find)
    

In [ ]:
get_non_simillar_sentences("П-07.22-6.1-КР-ТЧ.docx",True)

### Финальная результат

In [ ]:
# импортируем необходимые библиотеки

from docx import Document
from docx.shared import RGBColor
from docx.enum.text import WD_COLOR_INDEX

import numpy as np

from sentence_transformers import SentenceTransformer, util
import statistics as stat
from razdel import sentenize

In [ ]:
# загрузка данных из файла

def load_data_from_docx(file_name,include_sub=True):
    document = Document(file_name)
    words_to_delete = []
    for para in document.paragraphs:
        if para.style.name.startswith('Heading') and para.text.strip():
            words_to_delete.append(para.text)
    bold_under_to_delete = []

    # кусок по выборке подзаголовков
    if not include_sub:
        for paragraph in document.paragraphs:
            paragraph_text = ""
            for run in paragraph.runs:
                if run.bold and run.underline:
                    paragraph_text += run.text
            if paragraph_text:
                bold_under_to_delete.append(paragraph_text.strip())

    bold_under_to_delete.append("Таблица")

    def split_sentences(text):
        sentences = list(sentenize(text))
        sentence_texts = [i.text for i in sentences]
    
        combined_sentences = []
        temp_sentence = ""
    
        for i, sentence in enumerate(sentence_texts):
            temp_sentence += sentence
        
            # Проверка наличия непарных кавычек в предложении
            open_quotes = temp_sentence.count('"') % 2
        
            if open_quotes == 0:
                combined_sentences.append(temp_sentence.strip())
                temp_sentence = ""
            else:
                temp_sentence += " "
    
        # Если в конце осталась незакрытая кавычка
        if temp_sentence:
            combined_sentences.append(temp_sentence.strip())
    
        return combined_sentences
        
        
    # раскрытие заголовков
    def add_data_from_paragraph(rez_heads):
                para_sentences = split_sentences(paragraph.text)
                for sentence in para_sentences:
                    sentence = sentence.strip()
                    if sentence and (sentence not in bold_under_to_delete and any(word not in sentence for word in bold_under_to_delete)):
                        rez_heads.append(sentence)
                return rez_heads
    
    rez,rez_heads,found_start=[],[],False
    for paragraph in document.paragraphs:
        if any(word in paragraph.text for word in words_to_delete):
            found_start = True
            if rez_heads:
                rez.append(rez_heads)
                rez_heads = []
            
            # добавление заголовка
            rez_heads=add_data_from_paragraph(rez_heads)

        elif found_start:
            rez_heads=add_data_from_paragraph(rez_heads)
    
    if rez_heads:
        rez.append(rez_heads)
        
    return rez

In [ ]:
# расстояние каждого предложения до других (значения + сортировка)

def generate_seq_of_similarity_inc(data:list):
    results,results_values=[],[]
    embedder = SentenceTransformer('distiluse-base-multilingual-cased')
    sentence_embeddings = embedder.encode(data, convert_to_tensor=True)
    for i in range(len(data)):
        SENTENCE_NUM=i
        given_sentence_embedding = sentence_embeddings[SENTENCE_NUM]
        cosine_similarities_bert = util.pytorch_cos_sim(given_sentence_embedding.reshape(1, -1), sentence_embeddings).numpy().flatten()
        results_values.append(cosine_similarities_bert)

    data_outliner=np.mean(results_values)+1.9*np.std(results_values)


    for i, lst in enumerate(results_values):
        lst = lst
        mask = (lst > data_outliner) & (lst < 0.99)
        lst = lst[~mask]  
        results_values[i] = np.array(lst)
        results.append(np.argsort(lst))
    
    return results,results_values

In [ ]:
def get_frequence(results, results_values, data, isPrint=False,coeff=1.9):
    if len(results) == 1:
        #print(results)
        sorts = [i for i in results[0] if results_values[0][i] < np.median(results_values[0])]
        freq_dict = {val: sorts.count(val) for val in set(sorts)}
        sorted_freq_list = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
        data_stat = [i[1] for i in sorted_freq_list]
        if len(data_stat) == 0:
            threshold = 0
        else:
            threshold = round(stat.mean(data_stat) + coeff * stat.stdev(data_stat))
    else:
        sorts = []
        median = np.median(np.concatenate([np.array(inner) for inner in results_values]))
        for j in range(len(results)):
            sorts.append([i for i in results[j] if results_values[j][i] < median])
        flat_arr = [val for arr in sorts for val in arr]
        freq_dict = {val: flat_arr.count(val) for val in set(flat_arr)}
        sorted_freq_list = sorted(freq_dict.items(), key=lambda x: x[1], reverse=True)
        data_stat = [i[1] for i in sorted_freq_list]
        threshold = round(stat.mean(data_stat) + coeff * stat.stdev(data_stat))

    find, mark = [], False
    if isPrint:
        print("\nЧастота встречаемости элементов (статистический анализ):")
        mark = True
    for val, count in sorted_freq_list: 
        if count >= threshold:
            if mark:
                print(f"{val}: {count}")
                print(data[val])
            find.append(data[val])
    return find

In [ ]:
def get_review(file_name, find):
    document = Document(file_name)
    sentences_to_highlight = find

    for paragraph in document.paragraphs:
        for sentence in sentences_to_highlight:
            if sentence in paragraph.text:
            
                run = paragraph.add_run(sentence)
                run.font.color.rgb = RGBColor(255, 0, 0)
                run.font.highlight_color = WD_COLOR_INDEX.YELLOW

    document.save(f'{file_name}_review.docx')

In [ ]:
# итоговая функция

def get_non_simillar_sentences(filename,see_outliners=False,include_sub=True,coeff=1.5):
    data = load_data_from_docx(filename,include_sub)
    res,find=[],[]
    for i in data:
        res.append(generate_seq_of_similarity_inc(i))
        
    for i in range(len(res)):
        find.append(get_frequence(res[i][0],res[i][1],data[i],see_outliners,coeff))

    find=np.unique([item for sublist in find for item in sublist])
    get_review(filename,find)
    

In [ ]:
get_non_simillar_sentences("ПР_12_20_ПД_ПОС_Проект_организации_строительства.docx",include_sub=False)